<a href="https://colab.research.google.com/github/abhi-11nav/Movie-Sentiment-Analysis/blob/main/Movie_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing libraries 

import pandas as pd 
import numpy as np 

In [ ]:
# Loading the dataset into a variable 

dataset = pd.read_csv("/content/drive/MyDrive/IMDB Rating/movie.csv")

In [ ]:
dataset.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [ ]:
dataset.shape

(40000, 2)

In [ ]:
# Number of output classes their count. 

dataset["label"].value_counts()

0    20019
1    19981
Name: label, dtype: int64

## TEXT PRE-PROCESSING 

In [ ]:
import re 

import nltk 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
def clean_text(dataset, list_name):
  # Only retains alphabets (upper and lower case) in the text
  for i in range(dataset.shape[0]):
    list_name.append(re.sub('[^a-zA-Z]',' ',str(dataset.iloc[i,0])))
  print("Only alphabets remained in the text")

  # Organises the string spacing 
  for x in range(len(list_name)):
    list_name[x] = " ".join(x for x in str(list_name[x]).split()).lower()
  print("Text reorganized")

  for index in range(len(list_name)):
    temp_list = []
  #Lemmatization 
    for word in corpus[index].split():
      if word not in stopwords.words("english"):
        temp_list.append(word)
    corpus[index] = " ".join(lemmatizer.lemmatize(words) for words in temp_list )

In [ ]:
  corpus = []

clean_text(dataset, corpus)

Only alphabets remained in the text
Text reorganized


In [ ]:
corpus[0]                                                              

'grew b watching loving thunderbird mate school watched played thunderbird school lunch school wanted virgil scott one wanted alan counting became art form took child see movie hoping would get glimpse loved child bitterly disappointing high point snappy theme tune could compare original score thunderbird thankfully early saturday morning one television channel still play rerun series gerry anderson wife created jonatha frakes hand director chair version completely hopeless waste film utter rubbish cgi remake may acceptable replacing marionette homo sapiens subsp sapiens huge error judgment'

In [ ]:
from tqdm import tqdm 
from nltk import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
words_list = []

for index in tqdm(range(len(corpus))):
  words_list.append(word_tokenize(corpus[index]))

100%|██████████| 40000/40000 [00:17<00:00, 2239.48it/s]


# Training Word2Vec

In [ ]:
import gensim 

from gensim.models import Word2Vec

In [ ]:
model = gensim.models.Word2Vec(words_list, window=5, min_count = 2)

In [ ]:
X = []

for words in tqdm(words_list):
  X.append(np.mean([model.wv[word] for word in words if word in model.wv.index2word], axis=0))

100%|██████████| 40000/40000 [08:55<00:00, 74.68it/s]


In [ ]:
y = dataset["label"]
X = np.array(X)

In [ ]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X,y, train_size = 0.93, random_state= 101 )

# BUILDING THE MODEL 

In [ ]:
# Importing teh naive bayes classifier 

from sklearn.naive_bayes import GaussianNB

In [ ]:
gnb = GaussianNB()

In [ ]:
gnb.fit(train_X, train_y)

GaussianNB()

In [ ]:
# Evalutaion metric fro classification 

from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(test_y, gnb.predict(test_X))

0.7775

In [ ]:
import pickle 

In [ ]:
file = open("model.pkl", "wb")

pickle.dump(gnb, file)

In [ ]:
file2 = open("word2bec.pkl", "wb")

pickle.dump(model, file2)

In [ ]:
file.close()

In [ ]:
file2.close()